<a href="https://colab.research.google.com/github/ArmandDS/latin_news_web_scrapper/blob/master/web_scrapper_spanish_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for webscrapper news from 4 Latin America websites


Load the libraries


In [ ]:
import urllib.request as urlRequest
import urllib.parse as urlParse
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm


Define function to get the news articles from the website

In [ ]:
def get_article_content(url): 
  # pretend to be a chrome 47 browser on a windows 10 machine
  headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'}
  # create the url
  targetUrl = urlRequest.Request(url=url,headers=headers)

  # open the url
  x  = urlRequest.urlopen(targetUrl)

  # read the response
  response = x.read()
  return response

Create the dataframe to save the data

In [ ]:
df_news_elcomercio = pd.DataFrame(columns= ["title", "summary", "text", "link", "date", "name"])

## Now let´s run the scrapper to get the data from the newspaper: El comercio from Perú

In [ ]:
from datetime import datetime, timedelta
s = '2020-09-01'
count = 0
modified_date = datetime.strptime(s, "%Y-%m-%d")
for i in tqdm(range(0,30)):
  modified_date = modified_date - timedelta(days=1)
  url ="https://elcomercio.pe/archivo/todas/" + datetime.strftime(modified_date, "%Y-%m-%d")
  # print(url)
  response = get_article_content(url)
  soup = BeautifulSoup(response, 'html.parser')
  articles = soup.find_all("a", class_="story-item__title")
  link_aux=""
  if i%10 == 0:
    aux_print =1
  else:
    aux_print = 0
  
  for article in articles:
    link = article.get('href')
    if link and link!=link_aux and "mundo/" in link:
      # print(link)
      link_aux = link
      url = "https://elcomercio.pe" + link
      # print(url)
      try:
        response_article = get_article_content(url)
        soup2 = BeautifulSoup(response_article, 'html.parser')
        title = soup2.find_all("h1", class_="sht__title")[0].text
        # print(title)
        summary = soup2.find_all("h2", class_="sht__summary")[0].text
      except KeyboardInterrupt:
        break
      except Exception as e:
        continue
      text = ''
      for p in soup2.find_all("p", class_="story-contents__font-paragraph"):
        text = text + " " + p.text  
      df_news_elcomercio.at[count, 'title'] = title
      df_news_elcomercio.at[count, 'summary'] = summary
      df_news_elcomercio.at[count, 'text'] = text
      df_news_elcomercio.at[count, 'link'] = url  
      df_news_elcomercio.at[count, 'date'] = datetime.strftime(modified_date, "%Y-%m-%d")
      df_news_elcomercio.at[count, 'name'] = "el comercio"
      count +=1




    

100%|██████████| 30/30 [07:08<00:00, 14.27s/it]


In [ ]:
df_news_elcomercio.head()

,title,summary,text,link,date,name
0,"Coronavirus EN VIVO | Últimas noticias, casos...",Coronavirus EN VIVO en el mundo | Sigue aquí E...,Coronavirus EN VIVO | Sigue aquí EN DIRECTO l...,https://elcomercio.pe/mundo/coronavirus-en-viv...,2020-08-31,el comercio
1,"La peculiar historia de Sealand, la pequeña m...","Con una superficie de solo 0,004 km2 y ubicado...",Esta historia comienza con un correo electrón...,https://elcomercio.pe/mundo/europa/sealand-la-...,2020-08-31,el comercio
2,Ciudadanos chinos son obligados a tomar medic...,El gobierno de la región de Xinjiang está toma...,El gobierno de la región de Xinjiang está tom...,https://elcomercio.pe/mundo/asia/coronavirus-e...,2020-08-31,el comercio
3,"Venezuela: liberan a Roberto Marrero, jefe de...","El jefe del despacho del presidente interino, ...","Roberto Marrero, jefe del despacho del presid...",https://elcomercio.pe/mundo/venezuela/roberto-...,2020-08-31,el comercio
4,Venezuela: Maduro indulta a 110 dirigentes de...,"Fuera de la larga lista de indultados, se han ...",Las autoridades venezolanas comenzaron a exca...,https://elcomercio.pe/mundo/venezuela/venezuel...,2020-08-31,el comercio


## Get news from the newspaper: El Heraldo from México

In [ ]:
df_news_elheraldo = pd.DataFrame(columns= ["title", "summary", "text", "link", "date", "name"])

In [ ]:
url ="https://heraldodemexico.com.mx/seccion/orbe/"
response = get_article_content(url)
soup = BeautifulSoup(response, 'html.parser')

In [ ]:

for i in tqdm(range(17, 65)):
  url ="https://heraldodemexico.com.mx/seccion/orbe/pagina/"+ str(i) 
  # print(url)
  response = get_article_content(url)
  soup = BeautifulSoup(response, 'html.parser')
  columns = soup.find_all("article", class_="article__archive")
  # print(columns)
  if i%10 ==0:
    aux = 1
  else:
    aux = 0
  for col in columns:
    a_tag = col.find("a")
    if a_tag:
      link = a_tag.get("href")
#       print(link)
      response_article = get_article_content(link)
      soup2 = BeautifulSoup(response_article, 'html.parser')
  #     print(soup2)
      try:
        title = soup2.find_all("h1", class_="entry-title")[0].text
        summary = soup2.find_all("div", class_="article__singular--teaser")[0].text
        div_text= soup2.find("div", class_="article__singular--content clearfix")
        date  = soup2.find("time", class_="updated")
        date = date.get("datetime").split("T")[0]
        p_text  = div_text.find_all("p")
      except KeyboardInterrupt:
        break
      except:
        # print("Error")
        continue
      # if aux:
      #   print(title)
      text = ''
      for p in p_text:
        text = text + " "+ p.text
#       print(text)
      df_news_elheraldo.at[count, 'title'] = title
      df_news_elheraldo.at[count, 'summary'] = summary
      df_news_elheraldo.at[count, 'text'] = text
      df_news_elheraldo.at[count, 'link'] = link  
      df_news_elheraldo.at[count, 'date'] = date
      df_news_elheraldo.at[count, 'name'] = "el heraldo"
      count +=1

100%|██████████| 48/48 [17:37<00:00, 22.02s/it]


In [ ]:
df_news_elheraldo.head()

,title,summary,text,link,date,name
601,¡Todo un galán! Nieto de John F. Kennedy cauti...,El joven de 27 años planea seguir con el legad...,John “Jack” Schlossberg Kennedy llamó poderos...,https://heraldodemexico.com.mx/orbe/nieto-john...,2020-09-01,el heraldo
602,Dinamarca considerará violación actos sexuales...,El consentimiento debe darse de forma voluntar...,El Gobierno socialdemócrata danés y sus tres ...,https://heraldodemexico.com.mx/orbe/dinamarca-...,2020-09-01,el heraldo
603,Mark Zuckerberg dona 300 millones dólares para...,La donación de Zuckerberg y Chen -y su anuncio...,"El consejero delegado de Facebook, Mark Zucke...",https://heraldodemexico.com.mx/orbe/mark-zucke...,2020-09-01,el heraldo
604,¡Madre heroína! Desarma al secuestrador que qu...,La mujer no pudo evitar que se llevaran al peq...,"Leslie Bamaca paseaba a Mateo Alejandro, su h...",https://heraldodemexico.com.mx/orbe/madre-hero...,2020-09-01,el heraldo
605,¡Hay trabajo en Australia! Vive en una paradis...,Los interesados para esta vacante pueden inscr...,Si el sueño de tu vida siempre ha sido vivir ...,https://heraldodemexico.com.mx/orbe/trabajo-au...,2020-09-01,el heraldo


##Get data from another newspaper: 'El nuevo Siglo' from Colombia

In [ ]:
!pip install dateparser

     |████████████████████████████████| 368kB 3.0MB/s 


In [ ]:
url ="https://www.elnuevosiglo.com.co/seccion/internacional?page=2"
print(url)
response = get_article_content(url)
soup = BeautifulSoup(response, 'html.parser')


https://www.elnuevosiglo.com.co/seccion/internacional?page=2


In [ ]:
df_news_elnuevosiglo = pd.DataFrame(columns= ["title", "summary", "text", "link", "date", "name"])

In [ ]:
import dateparser
import datetime

for i in tqdm(range(9, 36)):
  url ="https://www.elnuevosiglo.com.co/seccion/internacional?page=" +str(i)
  # print(url)
  response = get_article_content(url)
  soup = BeautifulSoup(response, 'html.parser')
  columns = soup.find_all("article")
  if i%10 ==0:
    aux = 1
  else:
    aux = 0
  for col in columns:
    try:
      a_tag = col.find("a")
#       print(a_tag.get('href'))
      #Sumary   
      summary = col.find_all("div", class_="field--type-text-with-summary")[0].text
#       print(summary)
      link = 'https://www.elnuevosiglo.com.co/' + a_tag.get('href')
      response_article = get_article_content(link)
      soup2 = BeautifulSoup(response_article, 'html.parser')
      title = soup2.find_all("h1", class_="page-title")[0].text
      div_date = soup2.find_all("div", class_="info-line")[2].text
      date  = str(dateparser.parse(div_date, languages=[ 'es'],  date_formats=['%Y-%m-%d'])).split(" ")[0]
      div_text = soup2.find("div", class_="field--type-text-with-summary")
      p_text  = div_text.find_all("p")
      text = ''
      for p in p_text:
        text = text + " "+ p.text
      # if aux:
      #   print(title)
    except KeyboardInterrupt:
      break
    except:
      print("Error")
      continue   

    df_news_elnuevosiglo.at[count, 'title'] = title
    df_news_elnuevosiglo.at[count, 'summary'] = summary
    df_news_elnuevosiglo.at[count, 'text'] = text
    df_news_elnuevosiglo.at[count, 'link'] = link  
    df_news_elnuevosiglo.at[count, 'date'] = date
    df_news_elnuevosiglo.at[count, 'name'] = "el nuevo siglo"
    count +=1


100%|██████████| 27/27 [04:46<00:00, 10.63s/it]


In [ ]:
df_news_elnuevosiglo.tail()

,title,summary,text,link,date,name
1344,\n\nAlemania suspende su tratado de extradició...,Esto en reacción a la imposición por parte de ...,Alemania anunció el viernes que suspendía el ...,https://www.elnuevosiglo.com.co//articulos/07-...,2020-07-31,el nuevo siglo
1345,\n\nLos jóvenes no son invencibles al coronavi...,"El director general de la OMS, Tedros Adhanom ...",El director general de la Organización Mundia...,https://www.elnuevosiglo.com.co//articulos/07-...,2020-07-31,el nuevo siglo
1346,\n\nReino Unido aplaza desconfinamiento por au...,"El líder conservador Boris Johnson dijo, en co...",El gobierno británico decidió aplazar al meno...,https://www.elnuevosiglo.com.co//articulos/07-...,2020-07-31,el nuevo siglo
1347,\n\nVetadas candidaturas en Hong Kong\n,Doce candidatos prodemocracia de Hong Kong fue...,Doce candidatos prodemocracia de Hong Kong fu...,https://www.elnuevosiglo.com.co//articulos/07-...,2020-07-31,el nuevo siglo
1348,\n\n¿Aplazar las elecciones en Estados Unidos? \n,La sugerencia del mandatario de postergar las ...,"El presidente de los Estados Unidos, Donald T...",https://www.elnuevosiglo.com.co//articulos/07-...,2020-07-31,el nuevo siglo


## Now get the data from "la prensa grafica" from El Salvador

In [ ]:
url = 'https://www.laprensagrafica.com/ajax/get_section_news.html?viewmore=%2Fajax%2Fget_section_news.html&page=1&size=12&section=internacional'

print(url)
response = get_article_content(url)
soup = BeautifulSoup(response, 'html.parser')
article= soup.find_all("article")
print(len(article))

https://www.laprensagrafica.com/ajax/get_section_news.html?viewmore=%2Fajax%2Fget_section_news.html&page=1&size=12&section=internacional
12


In [ ]:
df_news_laprensagrafica = pd.DataFrame(columns= ["title", "summary", "text", "link", "date", "name"])

In [ ]:
count= 0

for i in tqdm(range(10, 36)):
  url ='https://www.laprensagrafica.com/ajax/get_section_news.html?viewmore=%2Fajax%2Fget_section_news.html&page='+str(i)+'&size=12&section=internacional'
  # print(url)
  response = get_article_content(url)
  soup = BeautifulSoup(response, 'html.parser')
  columns = soup.find_all("article")
  if i%10 ==0:
    aux = 1
  else:
    aux = 0
  for col in columns:
    try:
      a_tag = col.find("a")
  #       print(a_tag)
      link = 'https://www.laprensagrafica.com' + a_tag.get('href')
      response_article = get_article_content(link)
      soup2 = BeautifulSoup(response_article, 'html.parser')
      soup3 = soup2.find_all("article", class_='news-full')[0]
      title = soup3.find("h1", class_ = 'news-title' ).text.strip()
      summary= soup3.find("div", class_="news-excerpt").text.strip()
      date = soup3.find("time", class_="news-date").get('datetime').split("T")[0]
      div_text = soup3.find("div", class_="news-body")
      p_text  = div_text.find_all("p")
    except KeyboardInterrupt:
      break
    except:
      print("Error")
      continue  
    text = ''
    for p in p_text:
      text = text + " "+ p.text.strip()
    # if aux:
    #   print(title)
  
    df_news_laprensagrafica.at[count, 'title'] = title
    df_news_laprensagrafica.at[count, 'summary'] = summary
    df_news_laprensagrafica.at[count, 'text'] = text
    df_news_laprensagrafica.at[count, 'link'] = link  
    df_news_laprensagrafica.at[count, 'date'] = date
    df_news_laprensagrafica.at[count, 'name'] = "la prensa grafica"
    count +=1

100%|██████████| 26/26 [02:31<00:00,  5.85s/it]


In [ ]:
df_news_laprensagrafica.head()

,title,summary,text,link,date,name
0,Gobierno de Venezuela indulta a diputados opos...,"Entre los indultados, se encuentran los legisl...","El Gobierno de Venezuela indultó este lunes, ...",https://www.laprensagrafica.com/internacional/...,2020-08-31,la prensa grafica
1,FOTOS | Sale primer histórico vuelo comercial ...,El acuerdo de normalización de normalización e...,Un avión de El Al adornado con una Estrella d...,https://www.laprensagrafica.com/internacional/...,2020-08-31,la prensa grafica
2,Video: una niña quedó enganchada en un barrile...,Ocurrió en el Festival Internacional de Cometa...,"Pudo haber terminado en tragedia, pero afortu...",https://www.laprensagrafica.com/internacional/...,2020-08-30,la prensa grafica
3,Coronavirus: el mundo supera los 25 millones d...,El mayor número de casos se registran en Estad...,El número de casos confirmados de covid-19 ya...,https://www.laprensagrafica.com/internacional/...,2020-08-30,la prensa grafica
4,Trump y Biden se ven ganadores en recta final,La pandemia del covid-19 marca la campaña. Dem...,El presidente Donald Trump y el demócrata Joe...,https://www.laprensagrafica.com/internacional/...,2020-08-30,la prensa grafica


# Merge all the dataframe and save to it 

Shape of the each dataframe:

In [ ]:
df_news_elcomercio.shape

(601, 6)

In [ ]:
df_news_elheraldo.shape

(478, 6)

In [ ]:
df_news_elnuevosiglo.shape

(270, 6)

In [ ]:
df_news_laprensagrafica.shape

(287, 6)

Now, merge into 1:

In [ ]:
df_all_dataframes = pd.concat([ df_news_elcomercio, df_news_elheraldo, df_news_elnuevosiglo, df_news_laprensagrafica], ignore_index=True)
df_all_dataframes.shape

(1636, 6)

In [ ]:
df_all_dataframes.to_csv("df_all_dataframe.csv")

In [ ]:
from google.colab import files

files.download('df_all_dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>